In [2]:
import nltk
nltk.download('punkt')
from nltk.tokenize import word_tokenize, sent_tokenize
import pandas as pd
import string
from nltk.stem import PorterStemmer
from sklearn.model_selection import train_test_split
from nltk.tokenize.treebank import TreebankWordDetokenizer
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.utils import shuffle
from nltk.corpus import stopwords
import numpy
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.metrics import confusion_matrix

[nltk_data] Downloading package punkt to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package punkt is already up-to-date!


# Loading books

In [3]:
path = '../doyle' #insert path to Doyle's books

book_doyle_2 = open(path+'/baskerville_edited.txt', "r", encoding="utf-8")
text = book_doyle_2.read()
doyle_texts = text

book_doyle_3 = open(path+'/four_edited.txt', "r", encoding="utf-8")
text = book_doyle_3.read()
doyle_texts = doyle_texts + text

doyle_texts = doyle_texts.lower()
doyle_texts = doyle_texts.translate(str.maketrans('', '', string.punctuation))

In [4]:
words_doyle = word_tokenize(doyle_texts)
len(words_doyle)

108246

In [5]:
path2 = '../shaw' #insert path to Shaw's books
book_shaw_1 = open(path2+'/arms_edited.txt', "r", encoding="utf-8")
text = book_shaw_1.read()
shaw_texts = text

book_shaw_2 = open(path2+'/pygmalion_edited.txt', "r", encoding="utf-8")
text = book_shaw_2.read()
shaw_texts = shaw_texts + text

book_shaw_4 = open(path2+'/superman_edited.txt', "r", encoding="utf-8")
text = book_shaw_4.read()
shaw_texts = shaw_texts + text

shaw_texts = shaw_texts.lower()
shaw_texts = shaw_texts.translate(str.maketrans('', '', string.punctuation))

In [6]:
words_shaw = word_tokenize(shaw_texts)
len(words_shaw)

107499

In [7]:
words_doyle[:20]

['\ufeff',
 'mr',
 'sherlock',
 'holmes',
 'who',
 'was',
 'usually',
 'very',
 'late',
 'in',
 'the',
 'mornings',
 'save',
 'upon',
 'those',
 'not',
 'infrequent',
 'occasions',
 'when',
 'he']

In [8]:
words_shaw[:25]

['\ufeff',
 'night',
 'a',
 'ladys',
 'bedchamber',
 'in',
 'bulgaria',
 'in',
 'a',
 'small',
 'town',
 'near',
 'the',
 'dragoman',
 'pass',
 'it',
 'is',
 'late',
 'in',
 'november',
 'in',
 'the',
 'year',
 '1885',
 'and']

# Deleting unique words for each authors

In [9]:
words_doyle_unique = set(words_doyle)
words_shaw_unique = set(words_shaw)

In [10]:
len(words_doyle)

108246

In [11]:
len(words_doyle_unique)

8639

In [12]:
repeated = False
first_loop = True
for word in words_doyle:
    for word2 in words_shaw_unique:
        if word == word2:
            repeated = True
            break
    if repeated == True:
        if first_loop == False:
            text_doyle = text_doyle + ' ' + word
        else:
            text_doyle = word
            first_loop = False                                
    repeated = False

In [14]:
print(text_doyle)

﻿ mr sherlock holmes who was usually very late in the save upon those not occasions when he was up all night was seated at the breakfast table i stood upon the hearthrug and picked up the stick which our visitor had left behind him the night before it was a fine thick piece of wood of the sort which is known as a lawyer just under the head was a broad band nearly an inch across to james from his friends of the was engraved upon it with the date it was just such a stick as the oldfashioned family used to solid and reassuring well what do you make of it holmes was sitting with his back to me and i had given him no sign of my how did you know what i was doing i believe you have eyes in the back of your head i have at least a in front of me said he but tell me what do you make of our visitor stick since we have been so unfortunate as to miss him and have no notion of his this accidental souvenir becomes of importance let me hear you the man by an of it i think said i following as far as i 

In [318]:
text_doyle_token = word_tokenize(text_doyle)
type(text_doyle_token)
len(text_doyle_token)

92259

In [16]:
repeated = False
first_loop = True
for word in words_shaw:
    for word2 in words_doyle_unique:
        if word == word2:
            repeated = True
            break
    if repeated == True:
        if first_loop == False:
            text_shaw = text_shaw + ' ' + word
        else:
            text_shaw = word
            first_loop = False                                
    repeated = False

In [17]:
text_shaw_token = word_tokenize(text_shaw)
len(text_shaw_token)

91293

# Use of stemmer

In [18]:
stemmer = PorterStemmer()
text_shaw_stemmed = pd.DataFrame(text_shaw_token)
text_shaw_stemmed = text_shaw_stemmed.apply(lambda x: [stemmer.stem(y) for y in x])
text_shaw_stemmed = text_shaw_stemmed.values.flatten()

text_doyle_stemmed = pd.DataFrame(text_doyle_token)
text_doyle_stemmed = text_doyle_stemmed.apply(lambda x: [stemmer.stem(y) for y in x])
text_doyle_stemmed = text_doyle_stemmed.values.flatten()

In [19]:
type(text_doyle_stemmed)

numpy.ndarray

# Splitting texts for groups of n words and then to training and test sets

In [433]:
n = 30
groups_doyle = [text_doyle_token[i:i+n] for i in range(0, len(text_doyle_token), n)]
groups_shaw = [text_shaw_token[i:i+n] for i in range(0, len(text_shaw_token), n)]

In [434]:
len(groups_doyle)

3076

In [435]:
len(groups_shaw)

3044

In [436]:
groups_doyle[:1]

[['\ufeff',
  'mr',
  'sherlock',
  'holmes',
  'who',
  'was',
  'usually',
  'very',
  'late',
  'in',
  'the',
  'save',
  'upon',
  'those',
  'not',
  'occasions',
  'when',
  'he',
  'was',
  'up',
  'all',
  'night',
  'was',
  'seated',
  'at',
  'the',
  'breakfast',
  'table',
  'i',
  'stood']]

In [437]:
groups_shaw[:1]

[['\ufeff',
  'night',
  'a',
  'in',
  'in',
  'a',
  'small',
  'town',
  'near',
  'the',
  'pass',
  'it',
  'is',
  'late',
  'in',
  'november',
  'in',
  'the',
  'year',
  'and',
  'through',
  'an',
  'open',
  'window',
  'with',
  'a',
  'little',
  'balcony',
  'on',
  'the']]

In [438]:
groups_doyle_detoken = [TreebankWordDetokenizer().detokenize(groups_doyle[i]) for i in range(0, len(groups_doyle))]
groups_shaw_detoken = [TreebankWordDetokenizer().detokenize(groups_shaw[i]) for i in range(0, len(groups_shaw))]
train_doyle, test_doyle = train_test_split(groups_doyle_detoken, test_size=0.2)
train_shaw, test_shaw = train_test_split(groups_shaw_detoken, test_size=0.2)

In [439]:
df_train_doyle = pd.DataFrame(train_doyle)
df_train_doyle['Class'] = "Doyle"
df_train_doyle.rename(columns={0:'Group'},inplace=True)

df_train_shaw = pd.DataFrame(train_shaw)
df_train_shaw['Class'] = "Shaw"
df_train_shaw.rename(columns={0:'Group'},inplace=True)

df_test_doyle = pd.DataFrame(test_doyle)
df_test_doyle['Class'] = "Doyle"
df_test_doyle.rename(columns={0:'Group'},inplace=True)

df_test_shaw = pd.DataFrame(test_shaw)
df_test_shaw['Class'] = "Shaw"
df_test_shaw.rename(columns={0:'Group'},inplace=True)

In [440]:
df_train_shaw

,Group,Class
0,affected to speak i must say i am surprised to...,Shaw
1,library disappointed but we are just going out...,Shaw
2,which man will die will be a idea when the lea...,Shaw
3,to and honest men can never exercise any real ...,Shaw
4,diamonds no i want no gold and no diamonds a g...,Shaw
...,...,...
2430,protection tanner its all right then not going...,Shaw
2431,he stops and looks at them as if they had appe...,Shaw
2432,look shocked you know better than any of us th...,Shaw
2433,of course you are you are never ill so glad to...,Shaw


In [441]:
df_train = pd.concat([df_train_doyle, df_train_shaw], join='outer', keys=None, ignore_index=True)
df_test = pd.concat([df_test_doyle, df_test_shaw], join='outer', keys=None, ignore_index=True)

In [442]:
df_train

,Group,Class
0,are in the and myself i know now that i cannot...,Doyle
1,evening and the wild chase down the she listen...,Doyle
2,and was silent for a few moments a it was he s...,Doyle
3,that they get five pounds if they can give yes...,Doyle
4,than later come now no lies what were you doin...,Doyle
...,...,...
4890,protection tanner its all right then not going...,Shaw
4891,he stops and looks at them as if they had appe...,Shaw
4892,look shocked you know better than any of us th...,Shaw
4893,of course you are you are never ill so glad to...,Shaw


# Training

In [443]:
count_vectorizer = CountVectorizer(binary=True, min_df=0.001, max_df=0.4)
term_matrix = count_vectorizer.fit_transform(df_train['Group'])

In [444]:
transformer = TfidfTransformer().fit(term_matrix)
term_matrix_idf = transformer.transform(term_matrix)

In [445]:
feature_names = count_vectorizer.get_feature_names()
print(feature_names[:10])

['aback', 'abandoned', 'able', 'about', 'above', 'absence', 'absolutely', 'absurd', 'accept', 'accident']


In [446]:
len(feature_names)

2272

In [447]:
clf = MultinomialNB()
clf.fit(term_matrix_idf, df_train['Class'])
clf.score(term_matrix_idf, df_train['Class'])

0.9252298263534219

# Testing

In [448]:
df_test_shuffle = shuffle(df_test)

In [449]:
df_test_shuffle

,Group,Class
476,have important and you must not walk off we sh...,Doyle
1020,call it if you i am an able man a strong man i...,Shaw
193,of which away until it into the slopes of the ...,Doyle
60,written in a hand he had suffered for years fr...,Doyle
1081,climate one who would dress thus for the middl...,Shaw
...,...,...
1215,putting down his papers a remarkable looking y...,Shaw
352,bank the cloud was within fifty yards of where...,Doyle
574,the of loud were audible from and the hall doo...,Doyle
268,the long series of his i as i walked back alon...,Doyle


In [450]:
term_matrix_test = count_vectorizer.transform(df_test_shuffle['Group'])

In [451]:
clf.score(term_matrix_test, df_test_shuffle['Class'])

0.9126530612244897

In [452]:
clftest=clf.predict(term_matrix_test)

In [453]:
len(df_test_shuffle)

1225

In [454]:
clftest[0:20]
clftest = clftest.tolist()

In [455]:
len(clftest)

1225

In [456]:
df_test_shuffle['Predict'] = clftest

C:\Users\filip\Anaconda3\lib\site-packages\ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  """Entry point for launching an IPython kernel.


In [457]:
df_test_shuffle[0:10]

,Group,Class,Predict
476,have important and you must not walk off we sh...,Doyle,Doyle
1020,call it if you i am an able man a strong man i...,Shaw,Shaw
193,of which away until it into the slopes of the ...,Doyle,Doyle
60,written in a hand he had suffered for years fr...,Doyle,Doyle
1081,climate one who would dress thus for the middl...,Shaw,Shaw
88,plain to the north here with the queer hills b...,Doyle,Doyle
356,important and tell you more about the and espe...,Doyle,Doyle
839,of than that in which his has landed him on th...,Shaw,Doyle
1121,men than me will call for my dust and touch me...,Shaw,Shaw
677,coffee to the table well the run away with you...,Shaw,Shaw


In [458]:
i = 0
count = 0
for pred in df_test_shuffle['Class']:
    if pred == clftest[i]:
        count = count + 1
    i = i + 1

In [459]:
count

1118

In [460]:
len(df_test_shuffle)

1225

In [461]:
count/len(df_test_shuffle)*100

91.26530612244898

# Stopwords influence

In [467]:
nltk.download('stopwords')

count_vectorizer_stop = CountVectorizer(binary=True, min_df=0.002, max_df=0.4, stop_words=stopwords.words('english'))
term_matrix_stop = count_vectorizer_stop.fit_transform(df_train['Group'])
feature_names_stop = count_vectorizer_stop.get_feature_names()
print(feature_names_stop[:10])

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\filip\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


['able', 'absurd', 'accept', 'according', 'account', 'acquaintance', 'across', 'act', 'action', 'active']


In [468]:
transformer_stop = TfidfTransformer().fit(term_matrix_stop)
term_matrix_stop_idf = transformer_stop.transform(term_matrix_stop)

In [469]:
clf_stop = MultinomialNB()
clf_stop.fit(term_matrix_stop_idf, df_train['Class'])
clf_stop.score(term_matrix_stop_idf, df_train['Class'])

0.898467824310521

In [470]:
term_matrix_stop = count_vectorizer_stop.transform(df_test_shuffle['Group'])
clf_stop.score(term_matrix_stop, df_test_shuffle['Class'])

0.8889795918367347

# Ngrams model influence

In [471]:
count_vectorizer_ngram = CountVectorizer(binary=True, min_df=0.001, max_df=0.3, ngram_range=(1, 2))
term_matrix_ngram = count_vectorizer_ngram.fit_transform(df_train['Group'])
feature_names_ngram = count_vectorizer_ngram.get_feature_names()
print(feature_names_ngram[:10])

['aback', 'abandoned', 'able', 'able to', 'about', 'about and', 'about her', 'about him', 'about his', 'about in']


In [472]:
len(feature_names_ngram)

6465

In [473]:
transformer_ngram = TfidfTransformer().fit(term_matrix_ngram)
term_matrix_ngram_idf = transformer_ngram.transform(term_matrix_ngram)

In [474]:
clf_ngram = MultinomialNB()
clf_ngram.fit(term_matrix_ngram_idf, df_train['Class'])
clf_ngram.score(term_matrix_ngram_idf, df_train['Class'])

0.9468845760980592

In [475]:
term_matrix_ngram = count_vectorizer_ngram.transform(df_test_shuffle['Group'])
clf_ngram.score(term_matrix_ngram, df_test_shuffle['Class'])

0.913469387755102

In [476]:
df_test_shuffle_arr = df_test_shuffle['Class'].to_list()
predict_ngram = clf_ngram.predict(term_matrix_ngram)
print(confusion_matrix(df_test_shuffle_arr, predict_ngram))

[[573  43]
 [ 63 546]]


In [477]:
precision_ngram = 570/(570+46)
precision_ngram

0.9253246753246753

In [478]:
recall_ngram = 570/(570+67)
recall_ngram

0.8948194662480377

In [501]:
f1_ngram = 2*precision_ngram*recall_ngram/(precision_ngram+recall_ngram)
f1_ngram

0.9098164405426975

# New books test + summary

In [480]:
book_doyle_test = open(path+'/adventures_edited.txt', "r", encoding="utf-8")
doyle_test = book_doyle_test.read()
doyle_test = doyle_test.lower()
doyle_test = doyle_test.translate(str.maketrans('', '', string.punctuation))

doyle_test = word_tokenize(doyle_test)
len(doyle_test)

111785

In [481]:
book_shaw_test = open(path2+'/major_edited.txt', "r", encoding="utf-8")
shaw_test = book_shaw_test.read()
shaw_test = shaw_test.lower()
shaw_test = shaw_test.translate(str.maketrans('', '', string.punctuation))

shaw_test = word_tokenize(shaw_test)
len(shaw_test)

32623

In [482]:
groups_doyle_test = [doyle_test[i:i+n] for i in range(0, len(doyle_test), n)]
groups_doyle_test_detoken = [TreebankWordDetokenizer().detokenize(groups_doyle_test[i]) for i in range(0, len(groups_doyle_test))]
df_doyle_test = pd.DataFrame(groups_doyle_test_detoken)
df_doyle_test['Class'] = "Doyle"
df_doyle_test.rename(columns={0:'Group'},inplace=True)
df_doyle_test

,Group,Class
0,﻿to sherlock holmes she is always the woman i ...,Doyle
1,sex it was not that he felt any emotion akin t...,Doyle
2,mind he was i take it the most perfect reasoni...,Doyle
3,position he never spoke of the softer passions...,Doyle
4,motives and actions but for the trained reason...,Doyle
...,...,...
3722,broken man kept alive solely through the care ...,Doyle
3723,that he finds it difficult to part from them m...,Doyle
3724,the holder of a government appointment in the ...,Doyle
3725,once she had ceased to be the centre of one of...,Doyle


In [483]:
groups_shaw_test = [shaw_test[i:i+n] for i in range(0, len(shaw_test), n)]
groups_shaw_test_detoken = [TreebankWordDetokenizer().detokenize(groups_shaw_test[i]) for i in range(0, len(groups_shaw_test))]
df_shaw_test = pd.DataFrame(groups_shaw_test_detoken)
df_shaw_test['Class'] = "Shaw"
df_shaw_test.rename(columns={0:'Group'},inplace=True)
df_shaw_test

,Group,Class
0,﻿it is after dinner on a january night in the ...,Shaw
1,room upholstered in dark leather a person sitt...,Shaw
2,it a smaller writing table behind him on his l...,Shaw
3,the window is an armchair lady britomart is a ...,Shaw
4,well mannered and yet appallingly outspoken an...,Shaw
...,...,...
1083,lady britomart coming from the shed and stoppi...,Shaw
1084,learn to be independent and to act and think f...,Shaw
1085,lady britomarts ribs with her finger tips and ...,Shaw
1086,very naughty children what do you want barbara...,Shaw


In [484]:
term_matrix_test_doyle = count_vectorizer.transform(df_doyle_test['Group']) #basic test
clf.score(term_matrix_test_doyle, df_doyle_test['Class'])

0.8408907968875772

In [485]:
term_matrix_test_shaw = count_vectorizer.transform(df_shaw_test['Group']) #basic test
clf.score(term_matrix_test_shaw, df_shaw_test['Class'])

0.8621323529411765

In [486]:
term_matrix_test_doyle = count_vectorizer_stop.transform(df_doyle_test['Group']) #stop test
clf_stop.score(term_matrix_test_doyle, df_doyle_test['Class'])

0.7880332707271264

In [487]:
term_matrix_test_shaw = count_vectorizer_stop.transform(df_shaw_test['Group']) #stop test
clf_stop.score(term_matrix_test_shaw, df_shaw_test['Class'])

0.8382352941176471

In [488]:
term_matrix_test_doyle = count_vectorizer_ngram.transform(df_doyle_test['Group']) #ngram test
clf_ngram.score(term_matrix_test_doyle, df_doyle_test['Class'])

0.8438422323584652

In [489]:
term_matrix_test_shaw = count_vectorizer_ngram.transform(df_shaw_test['Group']) #ngram test
clf_ngram.score(term_matrix_test_shaw, df_shaw_test['Class'])

0.8676470588235294

In [490]:
df_new_books = pd.concat([df_shaw_test, df_doyle_test], join='outer', keys=None, ignore_index=True)
df_new_books

,Group,Class
0,﻿it is after dinner on a january night in the ...,Shaw
1,room upholstered in dark leather a person sitt...,Shaw
2,it a smaller writing table behind him on his l...,Shaw
3,the window is an armchair lady britomart is a ...,Shaw
4,well mannered and yet appallingly outspoken an...,Shaw
...,...,...
4810,broken man kept alive solely through the care ...,Doyle
4811,that he finds it difficult to part from them m...,Doyle
4812,the holder of a government appointment in the ...,Doyle
4813,once she had ceased to be the centre of one of...,Doyle


In [491]:
term_matrix_new_books = count_vectorizer.transform(df_new_books['Group']) #basic test zbiorczy
clf.score(term_matrix_new_books, df_new_books['Class'])

0.8456905503634475

In [492]:
term_matrix_new_books = count_vectorizer_stop.transform(df_new_books['Group']) #stop test zbiorczy
clf_stop.score(term_matrix_new_books, df_new_books['Class'])

0.7993769470404984

In [514]:
term_matrix_new_books = count_vectorizer_ngram.transform(df_new_books['Group']) #ngram test zbiorczy
clf_ngram.score(term_matrix_new_books, df_new_books['Class'])

0.8492211838006231

In [494]:
df_new_books_arr = df_new_books['Class'].to_list()

In [495]:
df_new_books_arr[:10]

['Shaw',
 'Shaw',
 'Shaw',
 'Shaw',
 'Shaw',
 'Shaw',
 'Shaw',
 'Shaw',
 'Shaw',
 'Shaw']

In [496]:
#doyle -> positive
#shaw -> negative
tp = 0
fp = 0
tn = 0
fn = 0
for i in range(0, len(df_new_books_arr)):
    if df_new_books_arr[i] == 'Doyle' and predict[i] == 'Doyle':
        tp = tp + 1
    if df_new_books_arr[i] == 'Doyle' and predict[i] == 'Shaw':
        fp = fp + 1
    if df_new_books_arr[i] == 'Shaw' and predict[i] == 'Shaw':
        tn = tn + 1
    if df_new_books_arr[i] == 'Shaw' and predict[i] == 'Doyle':
        fn = fn + 1

score = [tp, fp, tn, fn]
score

[3132, 595, 945, 143]

In [497]:
print(confusion_matrix(df_new_books_arr, predict))

[[3132  595]
 [ 143  945]]


In [498]:
precision = 3132/(3132+595)
precision

0.8403541722565065

In [499]:
recall = 3132/(3132+143)
recall

0.9563358778625954

In [500]:
f1 = precision*recall*2/(precision + recall)
f1

0.8946015424164523

# Unique words of each author influence

In [519]:
groups_doyle_nofilter = [words_doyle[i:i+n] for i in range(0, len(words_doyle), n)]
groups_shaw_nofilter = [words_shaw[i:i+n] for i in range(0, len(words_shaw), n)]

groups_doyle_detoken_nofilter = [TreebankWordDetokenizer().detokenize(groups_doyle_nofilter[i]) for i in range(0, len(groups_doyle_nofilter))]
groups_shaw_detoken_nofilter = [TreebankWordDetokenizer().detokenize(groups_shaw_nofilter[i]) for i in range(0, len(groups_shaw_nofilter))]
train_doyle_nofilter, test_doyle_nofilter = train_test_split(groups_doyle_detoken_nofilter, test_size=0.2)
train_shaw_nofilter, test_shaw_nofilter = train_test_split(groups_shaw_detoken_nofilter, test_size=0.2)

df_train_doyle_nofilter = pd.DataFrame(train_doyle_nofilter)
df_train_doyle_nofilter['Class'] = "Doyle"
df_train_doyle_nofilter.rename(columns={0:'Group'},inplace=True)

df_train_shaw_nofilter = pd.DataFrame(train_shaw_nofilter)
df_train_shaw_nofilter['Class'] = "Shaw"
df_train_shaw_nofilter.rename(columns={0:'Group'},inplace=True)

df_test_doyle_nofilter = pd.DataFrame(test_doyle_nofilter)
df_test_doyle_nofilter['Class'] = "Doyle"
df_test_doyle_nofilter.rename(columns={0:'Group'},inplace=True)

df_test_shaw_nofilter = pd.DataFrame(test_shaw_nofilter)
df_test_shaw_nofilter['Class'] = "Shaw"
df_test_shaw_nofilter.rename(columns={0:'Group'},inplace=True)

df_train_nofilter = pd.concat([df_train_doyle_nofilter, df_train_shaw_nofilter], join='outer', keys=None, ignore_index=True)
df_test_nofilter = pd.concat([df_test_doyle_nofilter, df_test_shaw_nofilter], join='outer', keys=None, ignore_index=True)

count_vectorizer_ngram_nofilter = CountVectorizer(binary=True, min_df=0.001, max_df=0.3, ngram_range=(1, 2))
term_matrix_ngram_nofilter = count_vectorizer_ngram_nofilter.fit_transform(df_train_nofilter['Group'])
feature_names_ngram_nofilter = count_vectorizer_ngram_nofilter.get_feature_names()

transformer_ngram_nofilter = TfidfTransformer().fit(term_matrix_ngram_nofilter)
term_matrix_ngram_idf_nofilter = transformer_ngram_nofilter.transform(term_matrix_ngram_nofilter)

clf_ngram_nofilter = MultinomialNB()
clf_ngram_nofilter.fit(term_matrix_ngram_idf_nofilter, df_train_nofilter['Class'])
clf_ngram_nofilter.score(term_matrix_ngram_idf_nofilter, df_train_nofilter['Class'])

0.9638512339242267

In [523]:
term_matrix_ngram_nofilter = count_vectorizer_ngram_nofilter.transform(df_test_shuffle['Group'])
clf_ngram_nofilter.score(term_matrix_ngram_nofilter, df_test_shuffle['Class'])

0.929795918367347

In [521]:
term_matrix_test_doyle = count_vectorizer_ngram_nofilter.transform(df_doyle_test['Group']) #ngram test
clf_ngram_nofilter.score(term_matrix_test_doyle, df_doyle_test['Class'])

0.894821572310169

In [522]:
term_matrix_test_shaw = count_vectorizer_ngram_nofilter.transform(df_shaw_test['Group']) #ngram test
clf_ngram_nofilter.score(term_matrix_test_shaw, df_shaw_test['Class'])

0.859375

In [520]:
term_matrix_new_books = count_vectorizer_ngram_nofilter.transform(df_new_books['Group']) #ngram test zbiorczy
clf_ngram_nofilter.score(term_matrix_new_books, df_new_books['Class'])

0.8868120456905504